# Testing models

In [1]:
import sys
sys.path.append('src/')

from Models import deberta_nli, bart_nli
from Helpers import *

# create helper function to standardise order


## Label structure

deberta: ['contradiction', 'entailment', 'neutral']

bart: ['contradiction', 'neutral', 'entailment']


In [2]:
deberta_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

([['0.96940809', '0.03018436', '0.00040754']], ['contradiction'])

In [3]:
bart_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

([['0.93711895', '0.05809933', '0.00478182']], ['contradiction'])

In [4]:
get_random_samples("data/toxicbias_train.csv", 10)

,comment_text,identity_attack_x,bias,target,category,rationale
353,Really unbelievable how many apologists for th...,0.500000,neutral,NaN,none,NaN
3275,So you are a sane young black or nonwhite li...,0.575758,bias,"blacks,women,lgbt","race,gender,lgbtq","prejudice against blacks,women and lgbtq"
1825,Why do Indonesians care? Because Muslim? why d...,0.600000,bias,muslims,religion,prejudice against blacks
1184,ridiculous but its OK to let and encourage chi...,0.600000,bias,transgenders,lgbtq,prejudice against transgenders
3164,Is everyone here really that ignorant about th...,0.500000,bias,christians,religion,prejudice against christians
1668,White men blame women. Of all colours. White w...,0.550000,neutral,NaN,none,NaN
3950,Mental illness Wally.... REally.. Being a t...,0.257143,bias,canadians,race,prejudice against canadians
3432,hes a nice little black man nothing elsenothin...,0.560606,neutral,NaN,none,NaN
3836,He probably felt like he was personally includ...,0.525000,bias,blacks,race,prejudice towards blacks
7,Unfortunately too many people see giving someo...,0.500000,bias,christians,religion,prejudice against christians
